In [21]:
import pandas as pd

In [22]:
data = pd.read_csv("/home/fm-pc-lt-228/Desktop/covid_nepali_tweets/covid19_tweeter_dataset (copy).csv")

In [23]:
data.shape

(35400, 6)

In [24]:
data["Label"].value_counts()

 1    14998
-1    13606
 0     6796
Name: Label, dtype: int64

In [25]:
import torch
import numpy as np 
import pandas as pd 
from transformers import BertTokenizer, BertModel, BertForMaskedLM, AutoTokenizer, AutoModelForMaskedLM
from scipy.spatial.distance import cosine 
import tokenizers 
import pandas as pd 
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score
from nltk.corpus import stopwords
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
# import snowballstemmer
import numpy
import os 
import re
import json
import pickle 

In [26]:
model = AutoModelForMaskedLM.from_pretrained("Shushant/nepaliBERT", output_hidden_states = True, return_dict = True, output_attentions = True)

In [27]:
tokenizers = AutoTokenizer.from_pretrained("Shushant/nepaliBERT")

pickle.dump(model, open('bert_embeddings','wb'))

pickle.dump(tokenizers, open('bert_tokenizers','wb'))

model = pickle.load(open('bert_embeddings','rb'))
tokenizers= pickle.load(open('bert_tokenizers','rb'))

In [28]:
tokenizers.tokenize("के मौजुदा लोकतान्त्रिक व्यवस्था राज्य पुनःसंरचनासँग जोडिएका हिजोका सवालहरूलाई यथास्थितिमा छोडेर सबल होला?")

['क',
 'मौ',
 '##ज',
 '##दा',
 'लोक',
 '##तान',
 '##तर',
 '##िक',
 'वय',
 '##वस',
 '##था',
 'राज',
 '##य',
 'पन',
 '##ः',
 '##सर',
 '##चना',
 '##सग',
 'जोडिएका',
 'हिजोका',
 'सवाल',
 '##हर',
 '##लाई',
 'यथा',
 '##स',
 '##थिति',
 '##मा',
 'छोड',
 '##र',
 'सबल',
 'होला',
 '?']

In [29]:
def get_bert_embedding_sentence(input_sentence):
    md = model
    tokenizer = tokenizers
#     md = local_model
#     tokenizer = local_tokenizers
    marked_text = " [CLS] " + input_sentence + " [SEP] "
    tokenized_text = tokenizer.tokenize(marked_text)

    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(indexed_tokens) 
    
    # Convert inputs to Pytorch tensors
    tokens_tensors = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    
    with torch.no_grad():
        outputs = md(tokens_tensors, segments_tensors)
        # removing the first hidden state
        # the first state is the input state 

        hidden_states = outputs.hidden_states
#         print(hidden_states[-2])
        # second_hidden_states = outputs[2]
    # hidden_states has shape [13 x 1 x 22 x 768]

    # token_vecs is a tensor with shape [22 x 768]
#     token_vecs = hidden_states[-2][0]
    # get last four layers
#     last_four_layers = [hidden_states[i] for i in (-1,-2, -3,-4)]
    # cast layers to a tuple and concatenate over the last dimension
#     cat_hidden_states = torch.cat(tuple(last_four_layers), dim=-1)
#     print(cat_hidden_states.shape)
    token_vecs = hidden_states[-2][0]
    # take the mean of the concatenated vector over the token dimension
#     sentence_embedding = torch.mean(cat_hidden_states, dim=0).squeeze()

    # Calculate the average of all 22 token vectors.
    sentence_embedding = torch.mean(token_vecs, dim=0)
#     sentence_embedding = torch.mean(token_vecs, dim=1)
    return sentence_embedding.numpy()

In [30]:
data["Tweet"][0]

'अमेरिकामा कोभिड बाट एकै दिन चार हजारभन्दा बढीको मृत्यु'

In [31]:
data['word_embeddings'] = data['Tweet'].apply(get_bert_embedding_sentence)

In [32]:
data.to_csv("final_embeddings.csv")

In [35]:
data["word_embeddings"][364].shape

(768,)

In [36]:
X,y = data['word_embeddings'], data['Label']

In [37]:
train_X, test_X, train_y, test_y = train_test_split(X,y, test_size = 0.2, random_state = 420)

train_df,test_df = train_test_split(data, test_size = 0.2, random_state = 420)

train_df.to_csv("train.csv")

test_df.to_csv("test.csv")

In [38]:
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier

svm_grid = SVC()

param_grid = {'C': [0.1, 1, 10], 'gamma': [1e-3, 1e-4]}

svm_grid = GridSearchCV(svm_grid, param_grid=param_grid, cv=5, refit=True, verbose=1)

In [39]:
svm_grid_fit = svm_grid.fit(train_X.tolist(), train_y)


Fitting 5 folds for each of 6 candidates, totalling 30 fits


In [40]:
svm_grid_fit.best_estimator_

SVC(C=10, gamma=0.001)

In [41]:
svc = SVC(C=10, gamma=0.001,probability=True)

In [42]:
svc.fit(train_X.tolist(), train_y)
#svc.fit(train_X, train_y)

SVC(C=10, gamma=0.001, probability=True)

In [43]:
svc_pred = svc.predict(test_X.tolist())
# svc_pred = svc.predict(test_X)

In [44]:
print(confusion_matrix(test_y, svc_pred))

[[2255   67  499]
 [ 287  528  484]
 [ 512   74 2374]]


In [45]:
print(classification_report(test_y, svc_pred))

              precision    recall  f1-score   support

          -1       0.74      0.80      0.77      2821
           0       0.79      0.41      0.54      1299
           1       0.71      0.80      0.75      2960

    accuracy                           0.73      7080
   macro avg       0.74      0.67      0.69      7080
weighted avg       0.73      0.73      0.72      7080



In [47]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

logreg = LogisticRegression()
ovr = OneVsRestClassifier(logreg)

ovr.fit(train_X.tolist(), train_y)
ovr_acc = ovr.score(test_X.tolist(), test_y)

print("One vs Rest Logisitic Regression accuracy = ", ovr_acc)

/home/fm-pc-lt-228/anaconda3/envs/nepali/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/fm-pc-lt-228/anaconda3/envs/nepali/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-lea

One vs Rest Logisitic Regression accuracy =  0.7201977401129943


/home/fm-pc-lt-228/anaconda3/envs/nepali/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
%%time

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

ovr = LogisticRegression()

param_grid = {'C': [1, 10, 100, 1000] }

ovr_grid = GridSearchCV(ovr, param_grid=param_grid, cv=5, refit=True, verbose=1)
ovr_grid_fit = ovr_grid.fit(train_X.tolist(), train_y)

In [50]:

%%time
'''
    Naive Bayes BernoulliNB
'''

nb_bern_clf = BernoulliNB()
nb_bern_clf.fit(X=train_X.tolist(), y=train_y)
nb_bern_score = nb_bern_clf.score(test_X.tolist(), test_y)

print('BernoulliNB', nb_bern_score)




BernoulliNB 0.5991525423728814
CPU times: user 937 ms, sys: 1.64 s, total: 2.57 s
Wall time: 647 ms


In [52]:
%%time
'''
    KNN Classifier
'''

from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier()
knn_clf.fit(X=train_X.tolist(), y=train_y)
knn_score = knn_clf.score(test_X.tolist(), test_y)

print('KNN Classifier', knn_score)


KNN Classifier 0.6888418079096045
CPU times: user 1min 13s, sys: 58.5 s, total: 2min 12s
Wall time: 11.2 s


In [53]:
%%time
'''
    Perceptron
'''

from sklearn.linear_model import Perceptron

percept = Perceptron(alpha=0.00001, tol=1e-5, penalty='l2', random_state=1, max_iter=1000)
percept = percept.fit(X=train_X.tolist(), y=train_y)
percept_score = percept.score(test_X.tolist(), test_y)

print('Perceptron Classifier', percept_score)

Perceptron Classifier 0.626412429378531
CPU times: user 1.19 s, sys: 24.1 ms, total: 1.22 s
Wall time: 1.14 s


In [54]:
from sklearn.neural_network import MLPClassifier

mlp_lbfgs = MLPClassifier(solver='lbfgs', alpha=1e-5, tol=1e-5,
                    hidden_layer_sizes=(100, 50, 25), random_state=1,
                   max_iter=1000)
mlp_lbfgs.fit(X=train_X.tolist(), y=train_y)

mlp_lbfgs_score = mlp_lbfgs.score(test_X.tolist(), test_y)

print('MultiLayered Perceptron Classifier - lbfgs', mlp_lbfgs_score)

MultiLayered Perceptron Classifier - lbfgs 0.6885593220338984


/home/fm-pc-lt-228/anaconda3/envs/nepali/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [55]:
%%time
'''
    Bagging Classifier
'''

from sklearn.ensemble import BaggingClassifier

bagg = BaggingClassifier()



bagg.fit(X=train_X.tolist(), y=train_y)

bagg_score = bagg.score(test_X.tolist(), test_y)



NameError: name 'print_acc' is not defined

In [58]:
print('Bagging Classifier', bagg_score)

Bagging Classifier 0.6639830508474577


In [56]:
%%time
'''
    SGD Classifier
'''

from sklearn.linear_model import SGDClassifier

sgd = SGDClassifier(max_iter=2000, n_jobs=-1, class_weight='balanced')



sgd.fit(X=train_X.tolist(), y=train_y)

sgd_score = sgd.score(test_X.tolist(), test_y)




NameError: name 'print_acc' is not defined

In [57]:
print('SGD Classifier', sgd_score)

SGD Classifier 0.7148305084745763
